In [ ]:
# Install dependencies for this example
# Note: This does not include itkwidgets, itself
import sys
!{sys.executable} -m pip install -U itk itk-totalvariation

In [5]:
from urllib.request import urlretrieve
import os

import itk
import numpy as np

from itkwidgets import view
import itkwidgets

In [6]:
# Download data
# Source: https://data.broadinstitute.org/bbbc/BBBC024/
file_name = 'HL50_cell_line_c00_03_extraction.tif'
if not os.path.exists(file_name):
    url = 'https://data.kitware.com/api/v1/file/5b61f16c8d777f06857c1949/download'
    urlretrieve(url, file_name)

In [7]:
image = itk.imread(file_name, itk.F)

view(image, cmap=itkwidgets.cm.BrBG, annotations=False, vmax=800, ui_collapsed=True)

Viewer(annotations=False, cmap=['BrBG'], geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<…

In [17]:
# Segment the cells
smoothed = itk.prox_tv_image_filter(image, weights=50, maximum_number_of_iterations=3)

LabelMapType = itk.Image[itk.UC, 3]

threshold_filter = itk.MomentsThresholdImageFilter[type(smoothed), LabelMapType].New(smoothed)
threshold_filter.SetInsideValue(0)
threshold_filter.SetOutsideValue(1)
threshold_filter.Update()
threshold = threshold_filter.GetThreshold()
thresholded = threshold_filter.GetOutput()

In [19]:
view(label_image=thresholded)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_label_image=<itk.itkIm…

In [20]:
connected_components = itk.connected_component_image_filter(thresholded)

view(label_image=connected_components)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_label_image=<itk.itkIm…

In [22]:
# We can combine the label map with the intensity image, add names for the labels, etc.
names = [(0, 'Background'), (1, 'First cell'), (2, 'Second cell')]
viewer = view(smoothed,
              label_image=connected_components,
              label_image_names=names,
              label_image_blend=0.8,
              rotate=True,
              gradient_opacity=0.5,
              slicing_planes=True)
viewer

Viewer(geometries=[], gradient_opacity=0.5, interpolation=False, label_image_names=[(0, 'Background'), (1, 'Firs…

Clicking on a label in a slice will toggle its weight.
We can also set label weights programmically.

In [27]:
label_count = len(np.unique(connected_components))
label_weights = np.ones((label_count,), dtype=np.float32)
label_weights[1] = 0.1
label_weights[2] = 0.2
viewer.label_image_weights = label_weights

We can also combine the intensity images, weighted and blended with the label maps, with iso-surface geometry.